In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from darts import TimeSeries
from darts.models import (
    ExponentialSmoothing,
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
)
from darts.metrics import mape, smape
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

## dataset preparation

In [2]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

In [3]:
X_train = np.load('./data/X_train_surge_new.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge_new.npz')

In [4]:
# slp : pressure fileds 
# surge : surge values

# train
slp_train = X_train['slp']
t_slp_train = X_train['t_slp']

t_surge1_input_train = X_train['t_surge1_input']
t_surge2_input_train = X_train['t_surge2_input']

surge1_input_train = X_train['surge1_input']
surge2_input_train = X_train['surge2_input']

t_surge1_output_train = X_train['t_surge1_output']
t_surge2_output_train = X_train['t_surge2_output']

# test
slp_test = X_test['slp']
t_slp_test = X_test['t_slp']

t_surge1_input_test = X_test['t_surge1_input']
t_surge2_input_test = X_test['t_surge2_input']

surge1_input_test = X_test['surge1_input']
surge2_input_test = X_test['surge2_input']

t_surge1_output_test = X_test['t_surge1_output']
t_surge2_output_test = X_test['t_surge2_output']

## data preprocessing

In [54]:
from darts.datasets import AirPassengersDataset
from sklearn.preprocessing import MinMaxScaler
from darts.dataprocessing.transformers import Scaler
series = AirPassengersDataset().load()
print(len(series))
# scaler = MinMaxScaler(feature_range=(-1, 1))
# transformer = Scaler(scaler)
# series_transformed = transformer.fit_transform(series)

144


In [48]:
scaler = MinMaxScaler(feature_range=(-1, 1))
transformer = Scaler()

This process should be repeated for every dataset

In [67]:
d_1 = {
    'time' : pd.to_datetime(t_surge1_input_train.flatten(), unit='s'),
    'surge': surge1_input_train.flatten()
}

df_surge_1 = pd.DataFrame(
    data=d_1
)

# df_surge_1.info()

# df_surge_1.index

df_surge_1.set_index('time')
df_surge_1.index
# df_surge_1.plot(y='surge', use_index=True)
# df_surge_1.iloc[:1000].plot(y='surge', use_index=True)

RangeIndex(start=0, stop=55990, step=1)

### autoencoders lstm

##  training model

In [ ]:
input_chunk_length = 10 # number or timesteps input to the model
output_chunk_length = 10 # numbe of timesteps output of the model
n_epochs = 10

In [56]:
nbeats = NBEATSModel(
    input_chunk_length=24, output_chunk_length=12, n_epochs = 10, random_state = 15
)

In [61]:
nbeats.fit(
    df_surge_1, 
    verbose=True
)

TypeError: object of type 'float' has no len()

In [ ]:
eval_model(nbeats)